In [1]:
import pandas as pd
import numpy as np
from concat_stations import concat_files
import data_prep
from rf_optimize import evaluate
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


In [2]:
# Load data and concatenate stations
full_data = concat_files()

# Replcae NaNs with medians
full_data = data_prep.nan_to_median(full_data)

#data with features we want
new_full_data = full_data.drop(columns=['weekday','windMeanSpeed.m.s', 'windDirection.grades', 'windMaxSpeed.m.s','precipitation.l.m2', 'year', 'month'])

#list of features we want
df_features = [x for x in full_data.columns if x not in ['bikes','weekday','windMeanSpeed.m.s', 'windDirection.grades', 'windMaxSpeed.m.s','precipitation.l.m2', 'year', 'month']]

In [15]:
#testing our assignment for later use in for loop. "give me all rows were station is number x"
s201 = new_full_data[new_full_data['station'] == 201]


In [32]:
#populate a list of dataframes, each with a station

station_data = []
station_number = 201
while (station_number < 276):
    station_data.append(new_full_data[new_full_data['station'] == (station_number)])
    station_number = station_number + 1


In [46]:
#have a look at what an element in the station data list looks like
station_data[-10]

,station,latitude,longitude,numDocks,timestamp,day,hour,weekhour,isHoliday,temperature.C,relHumidity.HR,airPressure.mb,bikes_3h_ago,full_profile_3h_diff_bikes,full_profile_bikes,short_profile_3h_diff_bikes,short_profile_bikes,bikes
48425,266,39.492946,-0.401981,15,1.412114e+09,1,0,49,0,21.3,85.0,855.3,1.0,1.000000,3.000000,1.000000,3.000000,5.0
48426,266,39.492946,-0.401981,15,1.412118e+09,1,1,50,0,21.1,86.0,1000.6,1.0,1.000000,3.666667,1.000000,3.666667,6.0
48427,266,39.492946,-0.401981,15,1.412122e+09,1,2,51,0,20.9,86.0,880.6,1.0,1.000000,3.250000,1.000000,3.250000,5.0
48428,266,39.492946,-0.401981,15,1.412125e+09,1,3,52,0,20.4,88.0,859.8,2.0,0.000000,3.250000,0.000000,3.250000,5.0
48429,266,39.492946,-0.401981,15,1.412129e+09,1,4,53,0,20.3,87.0,898.1,2.0,-0.416667,3.250000,-0.416667,3.250000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49165,266,39.492946,-0.401981,15,1.414778e+09,31,19,116,0,20.2,80.0,1023.5,1.0,-0.250000,3.500000,-0.250000,3.500000,4.0
49166,266,39.492946,-0.401981,15,1.414782e+09,31,20,117,0,20.2,82.0,1023.9,2.0,-0.750000,2.750000,-0.750000,2.750000,4.0
49167,266,39.492946,-0.401981,15,1.414786e+09,31,21,118,0,20.0,83.0,1024.0,2.0,0.500000,3.250000,0.500000,3.250000,8.0
49168,266,39.492946,-0.401981,15,1.414789e+09,31,22,119,0,19.3,84.0,1024.4,4.0,0.000000,3.500000,0.000000,3.500000,7.0


In [126]:
params = {"max_depth":7, "min_samples_leaf":30, "min_samples_split":600, "n_estimators" : 1800, "random_state" :8, "subsample":0.6, "max_features" : 7, "learning_rate" : 0.001}



def train_and_test_model(stationdf):

    y = np.array(stationdf['bikes']) # array for target variable
    X = stationdf[df_features] # Features (exluding bikes)
    X = np.array(X) # Turn into numpy array
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state= 42)
    # Instantiate the model
    reg = ensemble.GradientBoostingRegressor(**params)
    reg.fit(X_train, y_train)
    mae = metrics.mean_absolute_error(y_test, reg.predict(X_test))
    station_num = stationdf.iloc[1,0]
    print("for station " + str(station_num) + ", the mean absolute error (MAE) on test set is: {:.4f}".format(mae))
    
    return mae

In [127]:

accum = 0

#train eight models and have a look at the average MAE
for station_df in station_data:
    if (station_df.iloc[1,0] < 209):
        mae = train_and_test_model(station_df)
        accum = accum + mae

print(accum/8)





for station 201, the mean absolute error (MAE) on test set is: 3.2533
for station 202, the mean absolute error (MAE) on test set is: 3.1047
for station 203, the mean absolute error (MAE) on test set is: 3.7752
for station 204, the mean absolute error (MAE) on test set is: 5.9906
for station 205, the mean absolute error (MAE) on test set is: 3.1374
for station 206, the mean absolute error (MAE) on test set is: 6.1522
for station 207, the mean absolute error (MAE) on test set is: 4.5299
for station 208, the mean absolute error (MAE) on test set is: 8.1521
4.7619211364538945


In [132]:
from sklearn.svm import SVR

def traintest_SVR(stationdf):

    y = np.array(stationdf['bikes']) # array for target variable
    X = stationdf[df_features] # Features (exluding bikes)
    X = np.array(X) # Turn into numpy array
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state= 42)
    # Instantiate the model
    reg = SVR()
    reg.fit(X_train, y_train)

    mae = metrics.mean_absolute_error(y_test, reg.predict(X_test))
    station_num = stationdf.iloc[1,0]
    print("for station " + str(station_num) + ", the mean absolute error (MAE) on test set is: {:.4f}".format(mae))
    
    return mae

In [133]:
# same as before but trying with svr  not gradient boosting

accum = 0

for station_df in station_data:
    if (station_df.iloc[1,0] < 209):
        mae = traintest_SVR(station_df)
        accum = accum + mae


print(accum/8)




for station 201, the mean absolute error (MAE) on test set is: 3.0283
for station 202, the mean absolute error (MAE) on test set is: 2.8711
for station 203, the mean absolute error (MAE) on test set is: 3.3406
for station 204, the mean absolute error (MAE) on test set is: 5.3631
for station 205, the mean absolute error (MAE) on test set is: 2.7385
for station 206, the mean absolute error (MAE) on test set is: 5.6594
for station 207, the mean absolute error (MAE) on test set is: 3.7888
for station 208, the mean absolute error (MAE) on test set is: 7.0679
4.232215947082877
